In [3]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [20]:
# load the dataset
dataframe = pandas.read_csv('international-airline-passengers.csv', usecols=[1], engine='python', skipfooter=3)
dataset = dataframe.values
dataset = dataset.astype('float32')

In [21]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [22]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

95 48


In [23]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [24]:
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [25]:
trainX

array([[0.01544401, 0.02702703, 0.05405405, 0.04826255, 0.03281853],
       [0.02702703, 0.05405405, 0.04826255, 0.03281853, 0.05984557],
       [0.05405405, 0.04826255, 0.03281853, 0.05984557, 0.08494207],
       [0.04826255, 0.03281853, 0.05984557, 0.08494207, 0.08494207],
       [0.03281853, 0.05984557, 0.08494207, 0.08494207, 0.06177607],
       [0.05984557, 0.08494207, 0.08494207, 0.06177607, 0.02895753],
       [0.08494207, 0.08494207, 0.06177607, 0.02895753, 0.        ],
       [0.08494207, 0.06177607, 0.02895753, 0.        , 0.02702703],
       [0.06177607, 0.02895753, 0.        , 0.02702703, 0.02123553],
       [0.02895753, 0.        , 0.02702703, 0.02123553, 0.04247104],
       [0.        , 0.02702703, 0.02123553, 0.04247104, 0.07142857],
       [0.02702703, 0.02123553, 0.04247104, 0.07142857, 0.05984557],
       [0.02123553, 0.04247104, 0.07142857, 0.05984557, 0.04054055],
       [0.04247104, 0.07142857, 0.05984557, 0.04054055, 0.08687258],
       [0.07142857, 0.05984557, 0.

In [26]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [27]:
model = Sequential()
model.add(LSTM(20, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
 - 1s - loss: 0.0455
Epoch 2/100
 - 0s - loss: 0.0120
Epoch 3/100
 - 0s - loss: 0.0093
Epoch 4/100
 - 0s - loss: 0.0075
Epoch 5/100
 - 0s - loss: 0.0064
Epoch 6/100
 - 0s - loss: 0.0056
Epoch 7/100
 - 0s - loss: 0.0050
Epoch 8/100
 - 0s - loss: 0.0047
Epoch 9/100
 - 0s - loss: 0.0044
Epoch 10/100
 - 0s - loss: 0.0043
Epoch 11/100
 - 0s - loss: 0.0042
Epoch 12/100
 - 0s - loss: 0.0041
Epoch 13/100
 - 0s - loss: 0.0041
Epoch 14/100
 - 0s - loss: 0.0040
Epoch 15/100
 - 0s - loss: 0.0038
Epoch 16/100
 - 0s - loss: 0.0036
Epoch 17/100
 - 0s - loss: 0.0038
Epoch 18/100
 - 0s - loss: 0.0036
Epoch 19/100
 - 0s - loss: 0.0034
Epoch 20/100
 - 0s - loss: 0.0034
Epoch 21/100
 - 0s - loss: 0.0034
Epoch 22/100
 - 0s - loss: 0.0033
Epoch 23/100
 - 0s - loss: 0.0033
Epoch 24/100
 - 0s - loss: 0.0030
Epoch 25/100
 - 0s - loss: 0.0031
Epoch 26/100
 - 0s - loss: 0.0031
Epoch 27/100
 - 0s - loss: 0.0030
Epoch 28/100
 - 0s - loss: 0.0029
Epoch 29/100
 - 0s - loss: 0.0030
Epoch 30/100
 - 0s - lo

In [28]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 21.28 RMSE
Test Score: 54.84 RMSE
